In [3]:
import pandas as pd
import numpy as np

In [4]:
metro_df = pd.read_csv('new_metro_latest.csv')

In [5]:
metro_df.head()

,station_code,line_code,active,station_name_en,station_name_th,latitude,longitude
0,A1,arl,True,Suvarnabhumi,สุวรรณภูมิ,13.697907,100.752249
1,A2,arl,True,Lat Krabang,ลาดกระบัง,13.727890,100.748624
2,A3,arl,True,Ban Thap Chang,บ้านทับช้าง,13.732826,100.691306
3,A4,arl,True,Hua Mak,หัวหมาก,13.738048,100.645178
4,A5,arl,True,Ramkhamhaeng,รามคำแหง,13.742990,100.600148


In [6]:
bs_filter = (
    metro_df['line_code'].str.contains('red', case=False, na=False) &
    (metro_df['station_name_en'] == 'Bang Sue')
)


In [7]:
metro_df.loc[bs_filter, 'active'] = True

In [8]:
metro_df[bs_filter].head()

,station_code,line_code,active,station_name_en,station_name_th,latitude,longitude
266,RE01,red_east,True,Bang Sue,บางซื่อ,NaN,NaN
273,RN01,red_north,True,Bang Sue,บางซื่อ,13.804644,100.542054
287,RS01,red_south,True,Bang Sue,บางซื่อ,NaN,NaN
310,RW01,red_west,True,Bang Sue,บางซื่อ,13.804644,100.542054


In [9]:
hubs_df = metro_df[['station_name_en', 'station_name_th', 'latitude', 'longitude']]

In [10]:
hubs_df.head()

,station_name_en,station_name_th,latitude,longitude
0,Suvarnabhumi,สุวรรณภูมิ,13.697907,100.752249
1,Lat Krabang,ลาดกระบัง,13.727890,100.748624
2,Ban Thap Chang,บ้านทับช้าง,13.732826,100.691306
3,Hua Mak,หัวหมาก,13.738048,100.645178
4,Ramkhamhaeng,รามคำแหง,13.742990,100.600148


In [11]:
hubs_df = (hubs_df
    .sort_values(by=['station_name_en', 'latitude'])
    .drop_duplicates(subset=['station_name_en'], keep='first', ignore_index=True)
    .assign(id=lambda x: x.index + 1)
)
hubs_df.head()

,station_name_en,station_name_th,latitude,longitude,id
0,11th Infantry Regiment,กรมทหารราบที่ 11,13.867560,100.591895,1
1,Ancient City,เมืองโบราณ,NaN,NaN,2
2,Ari,อารีย์,13.779850,100.544653,3
3,Asok,อโศก,13.737048,100.560355,4
4,Ban Chimpli,บ้านฉิมพลี,NaN,NaN,5


In [12]:
new_metro_df = metro_df
new_metro_df['id'] = metro_df.merge(right=hubs_df, on='station_name_en')['id']
new_metro_df.head()

,station_code,line_code,active,station_name_en,station_name_th,latitude,longitude,id
0,A1,arl,True,Suvarnabhumi,สุวรรณภูมิ,13.697907,100.752249,263
1,A2,arl,True,Lat Krabang,ลาดกระบัง,13.727890,100.748624,103
2,A3,arl,True,Ban Thap Chang,บ้านทับช้าง,13.732826,100.691306,8
3,A4,arl,True,Hua Mak,หัวหมาก,13.738048,100.645178,67
4,A5,arl,True,Ramkhamhaeng,รามคำแหง,13.742990,100.600148,190


In [15]:
stations_df = new_metro_df[['station_code', 'line_code', 'active', "id"]]
stations_df = (stations_df
               .rename(columns={'id': 'hub_id'})
               .assign(id = lambda x: x.index + 1))
stations_df.head()

,station_code,line_code,active,hub_id,id
0,A1,arl,True,263,1
1,A2,arl,True,103,2
2,A3,arl,True,8,3
3,A4,arl,True,67,4
4,A5,arl,True,190,5


In [16]:
hubs_df.to_csv('hubs.csv', encoding='utf-8-sig', index=False)
stations_df.to_csv('stations.csv', encoding='utf-8-sig', index=False)